In [1]:
from google.colab import drive
drive.mount("/content/drive")

import os
from pathlib import Path

PROJECT_NAME = "electronic-picture-rag-project"
PROJECT_DIR = Path("/content/drive/MyDrive") / PROJECT_NAME

# Klasör yapısı
dirs = [
    PROJECT_DIR / "data" / "raw",          # Kaggle zip vs.
    PROJECT_DIR / "data" / "images",       # İşlenecek/verilecek görseller
    PROJECT_DIR / "index",                 # faiss index + metadata
    PROJECT_DIR / "models",                # gerekirse model cache
    PROJECT_DIR / "src",                   # python dosyaları
    PROJECT_DIR / "runs",                  # log vs.
]

for d in dirs:
    d.mkdir(parents=True, exist_ok=True)

print("✅ PROJECT_DIR:", PROJECT_DIR)
for d in dirs:
    print(" -", d)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ PROJECT_DIR: /content/drive/MyDrive/electronic-picture-rag-project
 - /content/drive/MyDrive/electronic-picture-rag-project/data/raw
 - /content/drive/MyDrive/electronic-picture-rag-project/data/images
 - /content/drive/MyDrive/electronic-picture-rag-project/index
 - /content/drive/MyDrive/electronic-picture-rag-project/models
 - /content/drive/MyDrive/electronic-picture-rag-project/src
 - /content/drive/MyDrive/electronic-picture-rag-project/runs


In [2]:
!pip -q install -U pip

# Gradio-Pillow uyumu için Pillow'u sabitle
!pip -q uninstall -y pillow
!pip -q install "pillow>=10.0,<12.0"

!pip -q install -U "torch" "torchvision" "torchaudio"
!pip -q install -U "transformers" "accelerate" "sentencepiece"
!pip -q install -U "faiss-cpu" "tqdm"
!pip -q install -U "kaggle"
!pip -q install -U "gradio"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 88.5 MB/s eta 0:00:00


In [2]:
from google.colab import files
uploaded = files.upload()

import os, json
from pathlib import Path


kaggle_dir = Path.home() / ".kaggle"
kaggle_dir.mkdir(parents=True, exist_ok=True)


if "kaggle.json" in uploaded:
    with open(kaggle_dir / "kaggle.json", "wb") as f:
        f.write(uploaded["kaggle.json"])


!chmod 600 /root/.kaggle/kaggle.json

print("✅ Kaggle configured:", kaggle_dir / "kaggle.json")


Saving kaggle.json to kaggle (1).json
✅ Kaggle configured: /root/.kaggle/kaggle.json


In [3]:
from pathlib import Path

PROJECT_DIR = Path("/content/drive/MyDrive/electronic-picture-rag-project")
RAW_DIR = PROJECT_DIR / "data" / "raw"
RAW_DIR.mkdir(parents=True, exist_ok=True)

DATASET_SLUG = "aryaminus/electronic-components"

print("📥 Downloading:", DATASET_SLUG)

!kaggle datasets download -d {DATASET_SLUG} -p "{RAW_DIR}" --force

print("📂 RAW_DIR contents:")
for f in RAW_DIR.iterdir():
    print(" -", f.name)


📥 Downloading: aryaminus/electronic-components
Dataset URL: https://www.kaggle.com/datasets/aryaminus/electronic-components
License(s): GPL-2.0
 98% 132M/135M [00:07<00:00, 21.8MB/s]
100% 135M/135M [00:07<00:00, 18.1MB/s]
📂 RAW_DIR contents:
 - electronic-components.zip


In [4]:
import zipfile
from pathlib import Path

PROJECT_DIR = Path("/content/drive/MyDrive/electronic-picture-rag-project")
RAW_DIR = PROJECT_DIR / "data" / "raw"
EXTRACT_DIR = RAW_DIR / "extracted"
EXTRACT_DIR.mkdir(parents=True, exist_ok=True)

zip_path = list(RAW_DIR.glob("*.zip"))[0]
print("📦 ZIP:", zip_path)

with zipfile.ZipFile(zip_path, "r") as z:
    z.extractall(EXTRACT_DIR)

print("✅ Extracted to:", EXTRACT_DIR)


📦 ZIP: /content/drive/MyDrive/electronic-picture-rag-project/data/raw/electronic-components.zip
✅ Extracted to: /content/drive/MyDrive/electronic-picture-rag-project/data/raw/extracted


In [5]:
from pathlib import Path

PROJECT_DIR = Path("/content/drive/MyDrive/electronic-picture-rag-project")
EXTRACT_DIR = PROJECT_DIR / "data" / "raw" / "extracted"


items = list(EXTRACT_DIR.rglob("*"))
print("Total items:", len(items))

print("\n--- First 40 paths ---")
for p in items[:40]:
    print(p)


exts = {".jpg", ".jpeg", ".png", ".webp"}
imgs = [p for p in items if p.is_file() and p.suffix.lower() in exts]
print("\nImage count:", len(imgs))
print("Sample images:")
for p in imgs[:10]:
    print(" -", p)


Total items: 22054

--- First 40 paths ---
/content/drive/MyDrive/electronic-picture-rag-project/data/raw/extracted/images
/content/drive/MyDrive/electronic-picture-rag-project/data/raw/extracted/images/Bypass-capacitor
/content/drive/MyDrive/electronic-picture-rag-project/data/raw/extracted/images/Electrolytic-capacitor
/content/drive/MyDrive/electronic-picture-rag-project/data/raw/extracted/images/Integrated-micro-circuit
/content/drive/MyDrive/electronic-picture-rag-project/data/raw/extracted/images/LED
/content/drive/MyDrive/electronic-picture-rag-project/data/raw/extracted/images/PNP-transistor
/content/drive/MyDrive/electronic-picture-rag-project/data/raw/extracted/images/armature
/content/drive/MyDrive/electronic-picture-rag-project/data/raw/extracted/images/attenuator
/content/drive/MyDrive/electronic-picture-rag-project/data/raw/extracted/images/cartridge-fuse
/content/drive/MyDrive/electronic-picture-rag-project/data/raw/extracted/images/clip-lead
/content/drive/MyDrive/elect

In [6]:
import re, shutil
from pathlib import Path

PROJECT_DIR = Path("/content/drive/MyDrive/electronic-picture-rag-project")
SRC_ROOT = PROJECT_DIR / "data" / "raw" / "extracted" / "images"
DST_ROOT = PROJECT_DIR / "data" / "images"
DST_ROOT.mkdir(parents=True, exist_ok=True)

exts = {".jpg", ".jpeg", ".png", ".webp"}

def norm_label(name: str) -> str:
    # "Bypass-capacitor" -> "bypass_capacitor"
    s = name.strip().lower()
    s = s.replace("&", "and")
    s = re.sub(r"\s+", "_", s)
    s = re.sub(r"[-]+", "_", s)
    s = re.sub(r"[^a-z0-9_]+", "", s)
    s = re.sub(r"_+", "_", s).strip("_")
    return s

# 1) Kaynak label klasörlerini bul
label_dirs = [p for p in SRC_ROOT.iterdir() if p.is_dir()]

print("Found label dirs:", len(label_dirs))

copied = 0
skipped = 0

for ld in sorted(label_dirs):
    # "images" gibi ara klasörleri atla/özel ele al
    if ld.name.lower() == "images":
        # içindeki alt klasörler varsa onları label gibi işle
        nested = [p for p in ld.iterdir() if p.is_dir()]
        for n in nested:
            label = norm_label(n.name)
            out_dir = DST_ROOT / label
            out_dir.mkdir(parents=True, exist_ok=True)

            for img_path in n.rglob("*"):
                if img_path.is_file() and img_path.suffix.lower() in exts:
                    dst = out_dir / img_path.name
                    if dst.exists():
                        skipped += 1
                        continue
                    shutil.copy2(img_path, dst)
                    copied += 1
        continue

    label = norm_label(ld.name)
    out_dir = DST_ROOT / label
    out_dir.mkdir(parents=True, exist_ok=True)

    for img_path in ld.rglob("*"):
        if img_path.is_file() and img_path.suffix.lower() in exts:
            dst = out_dir / img_path.name
            if dst.exists():
                skipped += 1
                continue
            shutil.copy2(img_path, dst)
            copied += 1

print("✅ Copied:", copied)
print("↩️ Skipped (already existed):", skipped)
print("DST_ROOT:", DST_ROOT)


Found label dirs: 37
✅ Copied: 10990
↩️ Skipped (already existed): 10990
DST_ROOT: /content/drive/MyDrive/electronic-picture-rag-project/data/images


In [7]:
from pathlib import Path

DST_ROOT = Path("/content/drive/MyDrive/electronic-picture-rag-project/data/images")
exts = {".jpg", ".jpeg", ".png", ".webp"}

labels = [p for p in DST_ROOT.iterdir() if p.is_dir()]
img_count = 0
for ld in labels:
    img_count += sum(1 for f in ld.rglob("*") if f.is_file() and f.suffix.lower() in exts)

print("✅ Label count:", len(labels))
print("✅ Total images:", img_count)

print("\nSample labels:")
for p in sorted(labels)[:20]:
    print(" -", p.name)


✅ Label count: 36
✅ Total images: 10990

Sample labels:
 - armature
 - attenuator
 - bypass_capacitor
 - cartridge_fuse
 - clip_lead
 - electric_relay
 - electrolytic_capacitor
 - filament
 - heat_sink
 - induction_coil
 - integrated_micro_circuit
 - jumper_cable
 - junction_transistor
 - led
 - light_circuit
 - limiter_clipper
 - local_oscillator
 - memory_chip
 - microchip
 - microprocessor


In [8]:
import shutil
from pathlib import Path

DST_ROOT = Path("/content/drive/MyDrive/electronic-picture-rag-project/data/images")

if DST_ROOT.exists():
    shutil.rmtree(DST_ROOT)

DST_ROOT.mkdir(parents=True, exist_ok=True)
print("✅ Cleaned:", DST_ROOT)


✅ Cleaned: /content/drive/MyDrive/electronic-picture-rag-project/data/images


In [9]:
import re, shutil
from pathlib import Path

PROJECT_DIR = Path("/content/drive/MyDrive/electronic-picture-rag-project")
SRC_ROOT = PROJECT_DIR / "data" / "raw" / "extracted" / "images"
DST_ROOT = PROJECT_DIR / "data" / "images"
DST_ROOT.mkdir(parents=True, exist_ok=True)

exts = {".jpg", ".jpeg", ".png", ".webp"}

def norm_label(name: str) -> str:
    s = name.strip().lower()
    s = s.replace("&", "and")
    s = re.sub(r"\s+", "_", s)
    s = re.sub(r"[-]+", "_", s)
    s = re.sub(r"[^a-z0-9_]+", "", s)
    s = re.sub(r"_+", "_", s).strip("_")
    return s

label_dirs = [p for p in SRC_ROOT.iterdir() if p.is_dir()]

copied = 0
skipped_dir = 0

for ld in sorted(label_dirs):
    # 🔥 Burada "images" isimli DUPLICATE klasörü tamamen atlıyoruz
    if ld.name.lower() == "images":
        skipped_dir += 1
        continue

    label = norm_label(ld.name)
    out_dir = DST_ROOT / label
    out_dir.mkdir(parents=True, exist_ok=True)

    for img_path in ld.rglob("*"):
        if img_path.is_file() and img_path.suffix.lower() in exts:
            dst = out_dir / img_path.name
            shutil.copy2(img_path, dst)
            copied += 1

print("✅ Copied:", copied)
print("🚫 Skipped duplicate top folder count:", skipped_dir)
print("DST_ROOT:", DST_ROOT)


✅ Copied: 10990
🚫 Skipped duplicate top folder count: 1
DST_ROOT: /content/drive/MyDrive/electronic-picture-rag-project/data/images


In [10]:
from pathlib import Path

DST_ROOT = Path("/content/drive/MyDrive/electronic-picture-rag-project/data/images")
exts = {".jpg", ".jpeg", ".png", ".webp"}

labels = [p for p in DST_ROOT.iterdir() if p.is_dir()]
img_count = 0
for ld in labels:
    img_count += sum(1 for f in ld.rglob("*") if f.is_file() and f.suffix.lower() in exts)

print("✅ Label count:", len(labels))
print("✅ Total images:", img_count)

print("\nSample labels:")
for p in sorted(labels)[:20]:
    print(" -", p.name)


✅ Label count: 36
✅ Total images: 10990

Sample labels:
 - armature
 - attenuator
 - bypass_capacitor
 - cartridge_fuse
 - clip_lead
 - electric_relay
 - electrolytic_capacitor
 - filament
 - heat_sink
 - induction_coil
 - integrated_micro_circuit
 - jumper_cable
 - junction_transistor
 - led
 - light_circuit
 - limiter_clipper
 - local_oscillator
 - memory_chip
 - microchip
 - microprocessor


In [11]:
import os, json
from pathlib import Path
from PIL import Image
import numpy as np
import torch
from tqdm import tqdm
import faiss
from transformers import CLIPProcessor, CLIPModel

PROJECT_DIR = Path("/content/drive/MyDrive/electronic-picture-rag-project")
DATA_DIR = PROJECT_DIR / "data" / "images"
INDEX_DIR = PROJECT_DIR / "index"
INDEX_DIR.mkdir(parents=True, exist_ok=True)

MODEL_ID = "openai/clip-vit-base-patch32"

device = "cuda" if torch.cuda.is_available() else "cpu"
print("✅ Device:", device)

# A100 için fp16
model = CLIPModel.from_pretrained(MODEL_ID, torch_dtype=torch.float16).to(device)
processor = CLIPProcessor.from_pretrained(MODEL_ID)

exts = {".jpg", ".jpeg", ".png", ".webp"}

def iter_images(data_dir: Path):
    for label_dir in sorted([p for p in data_dir.iterdir() if p.is_dir()]):
        label = label_dir.name
        for img_path in label_dir.rglob("*"):
            if img_path.is_file() and img_path.suffix.lower() in exts:
                yield img_path, label

@torch.inference_mode()
def embed_image(img: Image.Image) -> np.ndarray:
    inputs = processor(images=img, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}

    feats = model.get_image_features(**inputs)
    feats = feats / feats.norm(dim=-1, keepdim=True)
    feats = feats[0].detach().float().cpu().numpy().astype("float32")  # faiss float32 ister
    return feats

items = []
embs = []

all_imgs = list(iter_images(DATA_DIR))
print("🖼️ Total images to index:", len(all_imgs))
if len(all_imgs) == 0:
    raise RuntimeError("DATA_DIR boş görünüyor")

for img_path, label in tqdm(all_imgs, desc="Embedding"):
    try:
        img = Image.open(img_path).convert("RGB")
        vec = embed_image(img)
        items.append({"path": str(img_path), "label": label})
        embs.append(vec)
    except Exception as e:
        # bozuk dosya vs. varsa atla
        print("Skip:", img_path, "err:", e)

embs = np.stack(embs, axis=0)
dim = embs.shape[1]
print("✅ Emb matrix:", embs.shape)

# cosine için normalize ettik, inner product = cosine
index = faiss.IndexFlatIP(dim)
index.add(embs)

faiss.write_index(index, str(INDEX_DIR / "images.faiss"))
with open(INDEX_DIR / "items.json", "w", encoding="utf-8") as f:
    json.dump(items, f, ensure_ascii=False, indent=2)

print("✅ Saved:", INDEX_DIR / "images.faiss")
print("✅ Saved:", INDEX_DIR / "items.json")


`torch_dtype` is deprecated! Use `dtype` instead!


✅ Device: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

🖼️ Total images to index: 10990


Embedding: 100%|██████████| 10990/10990 [02:31<00:00, 72.36it/s]


✅ Emb matrix: (10990, 512)
✅ Saved: /content/drive/MyDrive/electronic-picture-rag-project/index/images.faiss
✅ Saved: /content/drive/MyDrive/electronic-picture-rag-project/index/items.json


In [18]:
import json
import numpy as np
from pathlib import Path
from PIL import Image
import torch
import faiss
from transformers import CLIPProcessor, CLIPModel
from collections import Counter

PROJECT_DIR = Path("/content/drive/MyDrive/electronic-picture-rag-project")
INDEX_DIR = PROJECT_DIR / "index"
MODEL_ID = "openai/clip-vit-base-patch32"

device = "cuda" if torch.cuda.is_available() else "cpu"
print("✅ Device:", device)

index = faiss.read_index(str(INDEX_DIR / "images.faiss"))
items = json.load(open(INDEX_DIR / "items.json", "r", encoding="utf-8"))

model = CLIPModel.from_pretrained(MODEL_ID, torch_dtype=torch.float16).to(device)
processor = CLIPProcessor.from_pretrained(MODEL_ID)

@torch.inference_mode()
def embed_query(img: Image.Image) -> np.ndarray:
    inputs = processor(images=img.convert("RGB"), return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}
    feats = model.get_image_features(**inputs)
    feats = feats / feats.norm(dim=-1, keepdim=True)
    return feats[0].detach().float().cpu().numpy().astype("float32")

from collections import Counter

def predict_label(image: Image.Image, k=5, score_threshold=0.28, vote_min=3):
    q = embed_query(image)[None, :]
    scores, idxs = index.search(q, k)

    results = []
    for s, i in zip(scores[0], idxs[0]):
        if i == -1:
            continue
        results.append({"score": float(s), "label": items[i]["label"], "path": items[i]["path"]})

    if not results:
        return "unknown", results, {"top": None, "vote": None, "vote_count": 0}

    top_score = results[0]["score"]
    labels = [r["label"] for r in results]
    vote_label, vote_count = Counter(labels).most_common(1)[0]

    # UNKNOWN kuralları
    if top_score < score_threshold:
        return "unknown", results, {"top": top_score, "vote": vote_label, "vote_count": vote_count}

    if vote_count < vote_min:
        return "unknown", results, {"top": top_score, "vote": vote_label, "vote_count": vote_count}

    return vote_label, results, {"top": top_score, "vote": vote_label, "vote_count": vote_count}






✅ Device: cuda


In [19]:
import gradio as gr

def gradio_fn(image):
    if image is None:
        return "No image", ""

    label, results, info = predict_label(image, k=5, score_threshold=0.28)

    lines = []
    lines.append(f"Decision: {label}")
    lines.append(f"top_score={info['top']:.3f} | vote={info['vote']} | vote_count={info['vote_count']}/5")
    lines.append("---- Top-5 ----")
    for r in results[:5]:
        lines.append(f"{r['label']} | score={r['score']:.3f}")

    return label, "\n".join(lines)

demo = gr.Interface(
    fn=gradio_fn,
    inputs=gr.Image(type="pil", label="Upload component image"),
    outputs=[gr.Textbox(label="Prediction"), gr.Textbox(label="Debug")],
    title="Electronic Picture RAG Project",
    description="CLIP embeddings + FAISS nearest neighbor. Returns UNKNOWN when confidence is low."
)

demo.launch(share=True)



Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://780097e586efb8e11f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [20]:
from pathlib import Path

PROJECT_DIR = Path("/content/drive/MyDrive/electronic-picture-rag-project")
SRC_DIR = PROJECT_DIR / "src"
SRC_DIR.mkdir(parents=True, exist_ok=True)

print("✅ SRC_DIR:", SRC_DIR)


✅ SRC_DIR: /content/drive/MyDrive/electronic-picture-rag-project/src


In [21]:
config_py = r'''
from pathlib import Path

PROJECT_NAME = "electronic-picture-rag-project"
PROJECT_DIR = Path(__file__).resolve().parents[1]

DATA_DIR = PROJECT_DIR / "data" / "images"
INDEX_DIR = PROJECT_DIR / "index"

MODEL_ID = "openai/clip-vit-base-patch32"

# Unknown karar eşikleri
SCORE_THRESHOLD = 0.28
VOTE_MIN = 3
TOPK = 5
'''

(Path("/content/drive/MyDrive/electronic-picture-rag-project/src/config.py")
 .write_text(config_py, encoding="utf-8"))

print("✅ wrote src/config.py")


✅ wrote src/config.py


In [22]:
build_index_py = r'''
import json
from pathlib import Path
from PIL import Image
import numpy as np
import torch
from tqdm import tqdm
import faiss
from transformers import CLIPProcessor, CLIPModel

from config import DATA_DIR, INDEX_DIR, MODEL_ID

exts = {".jpg", ".jpeg", ".png", ".webp"}

def iter_images(data_dir: Path):
    for label_dir in sorted([p for p in data_dir.iterdir() if p.is_dir()]):
        label = label_dir.name
        for img_path in label_dir.rglob("*"):
            if img_path.is_file() and img_path.suffix.lower() in exts:
                yield img_path, label

@torch.inference_mode()
def main():
    INDEX_DIR.mkdir(parents=True, exist_ok=True)

    device = "cuda" if torch.cuda.is_available() else "cpu"
    print("Device:", device)

    model = CLIPModel.from_pretrained(MODEL_ID, torch_dtype=torch.float16).to(device)
    processor = CLIPProcessor.from_pretrained(MODEL_ID)

    def embed_image(img: Image.Image) -> np.ndarray:
        inputs = processor(images=img, return_tensors="pt")
        inputs = {k: v.to(device) for k, v in inputs.items()}
        feats = model.get_image_features(**inputs)
        feats = feats / feats.norm(dim=-1, keepdim=True)
        return feats[0].detach().float().cpu().numpy().astype("float32")

    all_imgs = list(iter_images(DATA_DIR))
    print("Total images:", len(all_imgs))
    if len(all_imgs) == 0:
        raise RuntimeError(f"DATA_DIR empty: {DATA_DIR}")

    items = []
    embs = []

    for img_path, label in tqdm(all_imgs, desc="Embedding"):
        try:
            img = Image.open(img_path).convert("RGB")
            vec = embed_image(img)
            items.append({"path": str(img_path), "label": label})
            embs.append(vec)
        except Exception as e:
            print("Skip:", img_path, "err:", e)

    embs = np.stack(embs, axis=0)
    dim = embs.shape[1]
    print("Embeddings:", embs.shape)

    index = faiss.IndexFlatIP(dim)
    index.add(embs)

    faiss.write_index(index, str(INDEX_DIR / "images.faiss"))
    with open(INDEX_DIR / "items.json", "w", encoding="utf-8") as f:
        json.dump(items, f, ensure_ascii=False, indent=2)

    print("Saved:", INDEX_DIR / "images.faiss")
    print("Saved:", INDEX_DIR / "items.json")

if __name__ == "__main__":
    main()
'''

(Path("/content/drive/MyDrive/electronic-picture-rag-project/src/build_index.py")
 .write_text(build_index_py, encoding="utf-8"))

print("✅ wrote src/build_index.py")


✅ wrote src/build_index.py


In [23]:
app_py = r'''
import json
from pathlib import Path
from collections import Counter

import numpy as np
import torch
import faiss
import gradio as gr
from PIL import Image
from transformers import CLIPProcessor, CLIPModel

from config import INDEX_DIR, MODEL_ID, SCORE_THRESHOLD, VOTE_MIN, TOPK

device = "cuda" if torch.cuda.is_available() else "cpu"
print("✅ Device:", device)

index = faiss.read_index(str(INDEX_DIR / "images.faiss"))
items = json.load(open(INDEX_DIR / "items.json", "r", encoding="utf-8"))

model = CLIPModel.from_pretrained(MODEL_ID, torch_dtype=torch.float16).to(device)
processor = CLIPProcessor.from_pretrained(MODEL_ID)

@torch.inference_mode()
def embed_query(img: Image.Image) -> np.ndarray:
    inputs = processor(images=img.convert("RGB"), return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}
    feats = model.get_image_features(**inputs)
    feats = feats / feats.norm(dim=-1, keepdim=True)
    return feats[0].detach().float().cpu().numpy().astype("float32")

def predict_label(image: Image.Image, k=TOPK, score_threshold=SCORE_THRESHOLD, vote_min=VOTE_MIN):
    q = embed_query(image)[None, :]
    scores, idxs = index.search(q, k)

    results = []
    for s, i in zip(scores[0], idxs[0]):
        if i == -1:
            continue
        results.append({"score": float(s), "label": items[i]["label"], "path": items[i]["path"]})

    if not results:
        return "unknown", results, {"top": None, "vote": None, "vote_count": 0}

    top_score = results[0]["score"]
    labels = [r["label"] for r in results]
    vote_label, vote_count = Counter(labels).most_common(1)[0]

    if top_score < score_threshold:
        return "unknown", results, {"top": top_score, "vote": vote_label, "vote_count": vote_count}

    if vote_count < vote_min:
        return "unknown", results, {"top": top_score, "vote": vote_label, "vote_count": vote_count}

    return vote_label, results, {"top": top_score, "vote": vote_label, "vote_count": vote_count}

def gradio_fn(image):
    if image is None:
        return "No image", ""

    label, results, info = predict_label(image)

    lines = []
    lines.append(f"Decision: {label}")
    lines.append(f"top_score={info['top']:.3f} | vote={info['vote']} | vote_count={info['vote_count']}/{TOPK}")
    lines.append("---- Top-k ----")
    for r in results[:TOPK]:
        lines.append(f"{r['label']} | score={r['score']:.3f}")

    return label, "\n".join(lines)

demo = gr.Interface(
    fn=gradio_fn,
    inputs=gr.Image(type="pil", label="Upload component image"),
    outputs=[gr.Textbox(label="Prediction"), gr.Textbox(label="Debug")],
    title="Electronic Picture RAG Project",
    description="CLIP embeddings + FAISS nearest neighbor. Returns UNKNOWN when confidence is low."
)

if __name__ == "__main__":
    demo.launch(share=True)
'''
(Path("/content/drive/MyDrive/electronic-picture-rag-project/src/app.py")
 .write_text(app_py, encoding="utf-8"))

print("✅ wrote src/app.py")


✅ wrote src/app.py


In [24]:
req = """torch
torchvision
torchaudio
transformers
accelerate
sentencepiece
faiss-cpu
pillow>=10,<12
tqdm
gradio
kaggle
"""

(Path("/content/drive/MyDrive/electronic-picture-rag-project/requirements.txt")
 .write_text(req, encoding="utf-8"))

print("✅ wrote requirements.txt")


✅ wrote requirements.txt


In [29]:
from pathlib import Path

PROJECT_DIR = Path("/content/drive/MyDrive/electronic-picture-rag-project")
readme_path = PROJECT_DIR / "README.md"

metin = """# Electronic Picture RAG Project (Elektronik Parça Görsel RAG)

Bu proje, bir elektronik parça fotoğrafı yükleyince parçanın adını tahmin eder.
Yöntem:
- CLIP ile görsel embedding çıkarılır
- FAISS ile en benzer görseller bulunur
- Güven düşükse 'unknown' döner

## Klasör Yapısı

electronic-picture-rag-project/
- data/images/           -> Etiketlere göre ayrılmış görseller
- index/                 -> FAISS index dosyaları (images.faiss, items.json)
- src/
  - config.py
  - build_index.py       -> Index üretir (1 kere çalıştırılır)
  - app.py               -> Gradio arayüzünü açar
- requirements.txt       -> Gerekli kütüphaneler
- README.md              -> Bu dosya

## Kurulum

1) Kütüphaneleri kur:
pip install -r requirements.txt

## Index Oluşturma (1 kere)

Eğer index klasöründe images.faiss ve items.json yoksa:
python src/build_index.py

## Uygulamayı Çalıştırma

python src/app.py

Sonra Gradio arayüzü açılır, resim yükleyerek test edebilirsin.

## Notlar

- Colab restart olunca silinmemesi için proje Google Drive altında tutuluyor.
- Alakasız resimler için unknown dönmesi: top-k oylama + skor eşiği ile yapılır.
"""

readme_path.write_text(metin, encoding="utf-8")
print("✅ README.md oluşturuldu:", readme_path)


✅ README.md oluşturuldu: /content/drive/MyDrive/electronic-picture-rag-project/README.md


In [30]:
from pathlib import Path

PROJECT_DIR = Path("/content/drive/MyDrive/electronic-picture-rag-project")
for p in [
    PROJECT_DIR/"src/config.py",
    PROJECT_DIR/"src/build_index.py",
    PROJECT_DIR/"src/app.py",
    PROJECT_DIR/"requirements.txt",
    PROJECT_DIR/"README.md",
]:
    print("✅", p, "exists =", p.exists())

✅ /content/drive/MyDrive/electronic-picture-rag-project/src/config.py exists = True
✅ /content/drive/MyDrive/electronic-picture-rag-project/src/build_index.py exists = True
✅ /content/drive/MyDrive/electronic-picture-rag-project/src/app.py exists = True
✅ /content/drive/MyDrive/electronic-picture-rag-project/requirements.txt exists = True
✅ /content/drive/MyDrive/electronic-picture-rag-project/README.md exists = True


In [31]:
%cd /content/drive/MyDrive/electronic-picture-rag-project
!pwd
!ls -la


/content/drive/MyDrive/electronic-picture-rag-project
/content/drive/MyDrive/electronic-picture-rag-project
total 22
drwx------ 4 root root 4096 Dec 28 19:19 data
drwx------ 2 root root 4096 Dec 28 19:26 index
drwx------ 2 root root 4096 Dec 28 18:54 models
-rw------- 1 root root 1203 Dec 28 19:52 README.md
-rw------- 1 root root  111 Dec 28 19:47 requirements.txt
drwx------ 2 root root 4096 Dec 28 18:54 runs
drwx------ 2 root root 4096 Dec 28 19:47 src


In [33]:
from pathlib import Path

gitignore = """# Büyük veriler
data/
index/
runs/

# Python
__pycache__/
*.pyc

# Notebook çöpleri
.ipynb_checkpoints/

# OS
.DS_Store
Thumbs.db
"""

Path(".gitignore").write_text(gitignore, encoding="utf-8")
print("✅ .gitignore oluşturuldu")


✅ .gitignore oluşturuldu
